In [7]:
import os

# Create notes directory
os.makedirs("notes", exist_ok=True)

# Sample notes
notes = {
    "kubernetes.txt": """Kubernetes is an open-source system for automating deployment, scaling, and management of containerized applications. 
It works well with Docker and supports service discovery, load balancing, and rolling updates.""",

    "llm_research.txt": """Large Language Models (LLMs) such as GPT-4 and LLaMA are being used for Retrieval-Augmented Generation (RAG), 
chatbots, and AI agents. Key challenges include hallucinations, latency, and cost management.""",

    "travel.txt": """Last summer I traveled to Himachal Pradesh and visited Manali, Kasol, and Dharamshala. 
The mountains were beautiful, and the weather was much cooler compared to Delhi.""",

    "work_notes.txt": """Tasks for next sprint:
    - Finalize API Gateway migration to Gravitee
    - Implement Kafka DLQ handling
    - Review microservices scaling strategy
    - Prepare staff engineer interview case study"""
}

# Write notes to files
for fname, content in notes.items():
    with open(os.path.join("mcp_a/notes", fname), "w") as f:
        f.write(content)

print("Created sample notes:", os.listdir("mcp_a/notes"))


Created sample notes: ['travel.txt', 'kubernetes.txt', '.ipynb_checkpoints', 'work_notes.txt', 'llm_research.txt']


In [31]:
import json
from agents.mcp import MCPServerStdio

params = {"command": "python", "args": ["mcp_a/notes_server.py"]}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    # List tools (convert to dicts for JSON)
    tools = await server.list_tools()
    print("Available tools:")
    print(json.dumps([tool.model_dump() for tool in tools], indent=2))

    # Call list_notes
    notes = await server.call_tool("list_notes", {})
    print("\nNotes:")
    print(json.dumps(notes.model_dump(), indent=2))

    # Call read_note
    note_content = await server.call_tool("read_note", {"filename": "kubernetes.txt"})
    print("\nNote content:")
    print(json.dumps(note_content.model_dump(), indent=2))

    # Call search_notes
    search_result = await server.call_tool("search_notes", {"query": "himachal"})
    print("\nSearch results:")
    print(json.dumps(search_result.model_dump(), indent=2))


Available tools:
[
  {
    "name": "list_notes",
    "title": null,
    "description": "List all available notes in the notes folder.",
    "inputSchema": {
      "properties": {},
      "title": "list_notesArguments",
      "type": "object"
    },
    "outputSchema": null,
    "annotations": null,
    "meta": null
  },
  {
    "name": "read_note",
    "title": null,
    "description": "Read the contents of a note.",
    "inputSchema": {
      "properties": {
        "filename": {
          "title": "Filename",
          "type": "string"
        }
      },
      "required": [
        "filename"
      ],
      "title": "read_noteArguments",
      "type": "object"
    },
    "outputSchema": null,
    "annotations": null,
    "meta": null
  },
  {
    "name": "search_notes",
    "title": null,
    "description": "Search for a keyword in all notes.",
    "inputSchema": {
      "properties": {
        "query": {
          "title": "Query",
          "type": "string"
        }
      },
     

In [28]:
from openai import AsyncOpenAI
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from dotenv import load_dotenv
load_dotenv(override=True)

client = AsyncOpenAI()

instructions = """
You are a helpful notes assistant.
You can read, list, and search notes using the tools available.
"""

request = "Find anything about LLMs in my notes and show me the content."

params = {"command": "python", "args": ["mcp_a/notes_server.py"]}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="notes_agent",
        instructions=instructions,
        model="gpt-4.1-mini",  # Your OpenAI model
        mcp_servers=[mcp_server],
    )
    with trace("notes_agent"):
        result = await Runner.run(agent, request)
    print("\nFinal Output from Agent:\n")
    print(result.final_output)


[08/22/25 16:39:32] INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200   ]8;id=649262;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=791250;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             OK"                                                                                   

[08/22/25 16:39:34] INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200   ]8;id=15736;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=374333;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             OK"                                                                                   

[08/22/25 16:39:36] INFO     HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1   ]8;id=451469;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=296374;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py#1025\1025]8;;\
                             204 No Content"                                                                       

[08/22/25 16:39:37] INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200   ]8;id=640027;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=300647;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             OK"                                                                                   


Final Output from Agent:

Your notes mention that Large Language Models (LLMs) such as GPT-4 and LLaMA are being used for Retrieval-Augmented Generation (RAG), chatbots, and AI agents. Key challenges associated with these LLMs include hallucinations, latency, and cost management. If you want, I can help you find more detailed notes or related topics.


[08/22/25 16:39:41] INFO     HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1   ]8;id=584107;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=384544;file:///opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_client.py#1025\1025]8;;\
                             204 No Content"                                                                       